In [1]:
__depends__=[]
__dest__="../results/sample_table.tex"

# Make a Sample Table with Astropy
The [`astropy`](http://www.astropy.org/) package has some really nice dataframe-like functionality for constructing tables. These can even be exported to TeX as AASTeX `deluxetable` objects.

In [2]:
import numpy as np
import scipy.stats
import astropy.table
import astropy.io

First, construct a bunch of normal distributions of varying sample size.

In [3]:
sample_sizes = np.array([int(s) for s in np.logspace(1,4,4)])

In [4]:
true_mu,true_sigma = 1.,0.5
distributions = []
for s in sample_sizes:
    distributions.append(np.random.normal(loc=true_mu,scale=true_sigma,size=s))

Now, do a fit to each of the distributions.

In [5]:
fitted_mu,fitted_sigma = [],[]
for s,d in zip(sample_sizes,distributions):
    mu,sigma = scipy.stats.norm.fit(d)
    fitted_mu.append(mu)
    fitted_sigma.append(sigma)
fitted_mu = np.array(fitted_mu)
fitted_sigma = np.array(fitted_sigma)

Calculate the errors compared to the true mean.

In [6]:
mu_errors = np.fabs(fitted_mu - true_mu)/(fitted_mu + true_mu)*100.

And finally build a table from these results.

In [7]:
headers = (r'$N$',r'$\mu$',r'$\sigma$',r'$\varepsilon_{\mu}$')
results_table = astropy.table.Table(np.vstack((sample_sizes,fitted_mu,fitted_sigma,mu_errors)).T,names=headers)

In [8]:
results_table

$N$,$\mu$,$\sigma$,$\varepsilon_{\mu}$
float64,float64,float64,float64
10.0,1.08499278547,0.248521587445,4.07640669385
100.0,1.00397843772,0.534878480186,0.198526972201
1000.0,1.01259504987,0.518918236456,0.625811430219
10000.0,0.996547720865,0.501070333701,0.172912427741


Specify the output format and write a caption and then print it to a file using [`astropy.io.ascii`](http://docs.astropy.org/en/stable/io/ascii/index.html).

In [9]:
formats = {
    r'$N$':'%d',
    r'$\mu$':'.3f',r'$\sigma$':'.3f',r'$\varepsilon_{\mu}$':'.3f'
}
caption = r'''This table is generated on the fly in the AAS\TeX \texttt{deluxetable} style using AstroPy. It can be
easily viewed in the Jupyter notebook and is a great way for easily rebuilding tables each time your data changes.
'''

In [10]:
astropy.io.ascii.write(results_table, 
                       output=__dest__,
                       format='aastex',
                       formats=formats,
                       caption=caption
                      )